In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/NLP_P4

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/NLP_P4


In [2]:
%pip install vaderSentiment

     |████████████████████████████████| 133kB 3.5MB/s 


In [3]:
import os
import math
import time
import nltk
import random
import spacy
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import init
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from collections import Counter
from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

nltk.download('punkt')
get_pos = spacy.load("en_core_web_sm")
analyser = SentimentIntensityAnalyzer()
train_data = pd.read_csv('./gpu_train.csv', encoding='latin-1')
dev_data = pd.read_csv('./gpu_dev.csv', encoding='latin-1')
test_data = pd.read_csv('./gpu_test.csv', encoding='latin-1')
all_data = [train_data, dev_data, test_data]

PATH = './'

EMBED_DIM = 32
HIDDEN_DIM = 16
LAYERS = 1
EPOCHS = 100
LR = 3e-4
DEVICE = torch.device("cuda:0")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
def get_sentiment_scores(sentence):
    '''
    [negative, neutral, positive, compound]
    '''
    return [analyser.polarity_scores(sentence)[sentiment] \
            for sentiment in analyser.polarity_scores(sentence).keys()]

In [0]:
def get_character_n_grams(sentence, n):
    return [sentence[i:i+n] for i in range(len(sentence)-n+1)]

In [0]:
list_of_sentences = []
character_4_grams = []
def tag_pos(data):
    all_pos = []
    pos_counts = Counter()

    for df in data:
        df_pos = []
        for row in df.iterrows():
            row_pos = []
            for i in range(1, 7):
                parts_of_speech = get_pos(row[1][i])
                list_of_sentences.append([pos.text for pos in parts_of_speech])
                character_4_grams.append(get_character_n_grams(row[1][i], 4))
                sentence_pos = [pos.pos_ for pos in parts_of_speech]
                row_pos.append(sentence_pos)
                pos_counts.update(sentence_pos)
            df_pos.append(row_pos)
        all_pos.append(df_pos)
    return all_pos, pos_counts

In [0]:
pos_data = tag_pos(all_data)

In [8]:
model = Word2Vec(list_of_sentences, size=EMBED_DIM, min_count=1)
name = 'spacy_word2vec' + str(EMBED_DIM) + '.model'
model.save(name)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
## n-gram character 
model = Word2Vec(character_4_grams, size=EMBED_DIM, min_count=1)
name = 'n_gram_char_spacy_word2vec' + str(EMBED_DIM) + '.model'
model.save(name)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
w2v = 'spacy_word2vec' + str(EMBED_DIM) + '.model'
WORD2VEC = Word2Vec.load(w2v)
c2v = 'n_gram_char_spacy_word2vec' + str(EMBED_DIM) + '.model'
CHAR2VEC = Word2Vec.load(c2v)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def new_word_tokenize(sentence):
    return [pos.text for pos in get_pos(sentence)]

In [0]:
def get_one_hot(group, row, sentence, word):
    pos = pos_data[0][group][row][sentence][word] 
    return [1 if pos == list(pos_data[1].keys())[i] else 0 for i in range(len(pos_data[1].keys()))]

In [0]:
def create_vector(i, j, row, sentence, word, group):
    return np.array(list(WORD2VEC.wv[word]) + get_one_hot(group, row[0], i-1, j))

In [0]:
def new_embed(train_data, dev_data, test_data):
    training_data = [] 
    for row in train_data.iterrows():
        pos = [row[1][0]]
        neg = [row[1][0]]
        pos_n_grams = []
        neg_n_grams = []
        for i in range(1, 5):
            lst = [create_vector(i, j, row, row[1][i], word, 0) \
                    for j, word in enumerate(new_word_tokenize(row[1][i]))]
            pos.append(lst)
            neg.append(lst)
            n_gram_lst = [np.array(list(CHAR2VEC.wv[char])) for char in get_character_n_grams(row[1][i], 4)]
            pos_n_grams.append(n_gram_lst)
            neg_n_grams.append(n_gram_lst)               
        if row[1][7] == 1:
            pos.append([np.array(list(WORD2VEC.wv[word]) + get_one_hot(0, row[0], 4, j)) \
                   for j, word in enumerate(new_word_tokenize(row[1][5]))])
            neg.append([np.array(list(WORD2VEC.wv[word]) + get_one_hot(0, row[0], 5, j)) \
                   for j, word in enumerate(new_word_tokenize(row[1][6]))])
            pos_n_grams.append([np.array(list(CHAR2VEC.wv[char])) for char in get_character_n_grams(row[1][5], 4)])
            neg_n_grams.append([np.array(list(CHAR2VEC.wv[char])) for char in get_character_n_grams(row[1][6], 4)])
        elif row[1][7] == 2:
            pos.append([np.array(list(WORD2VEC.wv[word]) + get_one_hot(0, row[0], 5, j)) \
                   for j, word in enumerate(new_word_tokenize(row[1][6]))])
            neg.append([np.array(list(WORD2VEC.wv[word]) + get_one_hot(0, row[0], 4, j)) \
                   for j, word in enumerate(new_word_tokenize(row[1][5]))])
            pos_n_grams.append([np.array(list(CHAR2VEC.wv[char])) for char in get_character_n_grams(row[1][6], 4)])
            neg_n_grams.append([np.array(list(CHAR2VEC.wv[char])) for char in get_character_n_grams(row[1][5], 4)])
        pos.append(1)
        neg.append(0)
        training_data.append((pos, pos_n_grams))
        training_data.append((neg, neg_n_grams))
    
    development_data = []
    for row in dev_data.iterrows():
        ngrams = []
        sentences = [row[1][0]]
        for i in range(1, 7):
            lst = [create_vector(i, j, row, row[1][i], word, 1) \
                    for j, word in enumerate(new_word_tokenize(row[1][i]))]
            ngrams.append([np.array(list(CHAR2VEC.wv[char])) for char in get_character_n_grams(row[1][i], 4)])
            sentences.append(lst)
        sentences.append(row[1][7] - 1)
        development_data.append((sentences, ngrams))
        
    testing_data = []
    for row in test_data.iterrows():
        ngrams = []
        sentences = [row[1][0]]
        for i in range(1, 7):
            lst = [create_vector(i, j, row, row[1][i], word, 2) \
                    for j, word in enumerate(new_word_tokenize(row[1][i]))]
            ngrams.append([np.array(list(CHAR2VEC.wv[char])) for char in get_character_n_grams(row[1][i], 4)])
            sentences.append(lst)
        testing_data.append((sentences, ngrams))
        
    return training_data, development_data, testing_data

In [0]:
training_data, development_data, testing_data = new_embed(all_data[0], all_data[1], all_data[2])

In [0]:
class Blind(nn.Module):
    def __init__(self):
        super(Blind, self).__init__()
        self.gru = nn.GRU(EMBED_DIM+17, HIDDEN_DIM, LAYERS, batch_first=True, bidirectional=False)
        self.criterion = nn.NLLLoss()
        self.cuda(device=DEVICE)

    def compute_Loss(self, predicted_vector, gold_label):
        return self.criterion(predicted_vector, gold_label)

    def forward(self, data):
        input_vector = torch.tensor(data, device=DEVICE, dtype=torch.float).unsqueeze(dim=0)
        h_0 = torch.zeros((LAYERS, 1, HIDDEN_DIM), device=DEVICE)
        output, __ = self.gru(input_vector, h_0)
        x = output[0][-1]
        return x


In [0]:
class NSP(nn.Module):
    def __init__(self):
        super(NSP, self).__init__()
        self.beginning = nn.GRU(EMBED_DIM+17, HIDDEN_DIM, LAYERS, batch_first=True, bidirectional=False)
        self.ending = nn.GRU(EMBED_DIM+17, HIDDEN_DIM, LAYERS, batch_first=True, bidirectional=False)
        self.start_ngram = nn.GRU(EMBED_DIM, HIDDEN_DIM, LAYERS, batch_first=True, bidirectional=False)
        self.end_ngram = nn.GRU(EMBED_DIM, HIDDEN_DIM, LAYERS, batch_first=True, bidirectional=False)
        self.blind = Blind()
        self.l1 = nn.Linear(HIDDEN_DIM, 2)
        self.l2 = nn.Linear(HIDDEN_DIM, 2)
        self.l3 = nn.Linear(HIDDEN_DIM, 2)
        self.l4 = nn.Linear(25, 2)
        # self.linear = nn.Linear(HIDDEN_DIM*1+25, 2)
        self.softmax = nn.LogSoftmax(dim=0)
        self.criterion = nn.NLLLoss()
        self.optimizer = optim.Adam(self.parameters(), lr=LR)
        self.cuda(device=DEVICE)
        self.testing = False
        
    def setup(self, data):
        input_1 = torch.tensor(np.expand_dims(data[1] + data[2] + data[3] + data[4], axis=0), device=DEVICE, dtype=torch.float)
        input_2 = torch.tensor(np.expand_dims(data[5], axis=0), device=DEVICE, dtype=torch.float)
        df = all_data[0]
        try:
            row = df.loc[df['InputStoryid'] == data[0]].values[0]
        except: 
            df = all_data[1]
            row = df.loc[df['InputStoryid'] == data[0]].values[0]

        sentences = row[1:5]
        input_1_sentiment_scores = [get_sentiment_scores(sentence) + [len(sentence.split())] for sentence in sentences]
        if self.testing:
            correct_column = 5
        elif row[7] == 1 and data[6]:
            correct_column = 5
        elif row[7] == 2 and data[6]:
            correct_column = 6
        elif row[7] == 1 and not data[6]:
            correct_column = 6
        else:
            correct_column = 5
        input_2_sentiment = get_sentiment_scores(row[correct_column]) + [len(row[correct_column].split())]
        sentiment = list(np.array(input_1_sentiment_scores).flatten()) + input_2_sentiment 
        sentiment = torch.tensor(sentiment, device=DEVICE, dtype=torch.float)
        return input_1, input_2, sentiment

    def compute_Loss(self, predicted_vector, gold_label):
        return self.criterion(predicted_vector, gold_label)

    def forward(self, data):        
        input_1, input_2, sentiment = self.setup(data[0])
        input_3 = torch.tensor(np.expand_dims(data[1][0] + data[1][1] + data[1][2] + data[1][3], axis=0), device=DEVICE, dtype=torch.float)
        input_4 = torch.tensor(np.expand_dims(data[1][4], axis=0), device=DEVICE, dtype=torch.float)
        h_0 = torch.zeros((LAYERS, 1, HIDDEN_DIM), device=DEVICE)
        __, h_n = self.beginning(input_1, h_0)
        output, __ = self.ending(input_2, h_n)
        x = output[0][-1]
        x = self.softmax(self.l1(x))
        h_0 = torch.zeros((LAYERS, 1, HIDDEN_DIM), device=DEVICE)
        __, h_n = self.start_ngram(input_3, h_0)
        output, __ = self.end_ngram(input_4, h_n)
        y = output[0][-1]
        y = self.softmax(self.l2(y))
        b = self.blind(data[0][5])
        b = self.softmax(self.l3(b))
        s = self.softmax(self.l4(sentiment))
        # z = torch.cat((y, sentiment), 0)
        # z = self.linear(z)
        # z = self.softmax(z)
        return x + y + b + s

In [18]:
print('Initializing Model')
model = NSP()
prev_dev_acc = 0.0
for epoch in range(EPOCHS):
    checkpoint = PATH + '-e' + str((epoch + 1))
    model.train()
    model.optimizer.zero_grad()
    loss = None
    correct = 0
    total = 0
    start_time = time.time()
    print('Training started for epoch {}'.format(epoch + 1))
    random.shuffle(training_data)
    N = len(training_data)
    for index  in tqdm(range(N)):
        model.optimizer.zero_grad()
        input_vector = training_data[index]
        gold_label = input_vector[0][6]
        predicted_vector = model(input_vector)
        predicted_label = torch.argmax(predicted_vector)
        correct += int(predicted_label == gold_label)
        total += 1
        loss = model.compute_Loss(predicted_vector.view(1, -1), torch.tensor([gold_label], device=DEVICE))
        loss.backward()
        model.optimizer.step()
    print('Training accuracy for epoch {}: {}'.format(epoch + 1, correct / total))
    correct = 0
    total = 0
    start_time = time.time()
    random.shuffle(development_data)
    N = len(development_data)
    model.eval()
    model.optimizer.zero_grad()
    for index in tqdm(range(N)):
        sample = development_data[index]
        input_1 = (sample[0][0:6] + [sample[0][7]], sample[1][:5])
        input_2 = (sample[0][0:5] + sample[0][6:], sample[1][:4] + [sample[1][5]])
        gold_label = sample[0][7]
        prediction_1 = model(input_1)
        prediction_2 = model(input_2)
        prob_truthful_1 = prediction_1[1]
        prob_false_1 = prediction_1[0]
        prob_truthful_2 = prediction_2[1]
        prob_false_2 = prediction_2[0]
        probs = [prob_truthful_1, prob_false_1, prob_truthful_2, prob_false_2]
        max_index = probs.index(max(probs))
        if max_index == 0 or max_index == 3:
            predicted_label = 0
        if max_index == 1 or max_index == 2:
            predicted_label = 1
        correct += int(predicted_label == gold_label)
        total += 1
    dev_acc = correct / total
    if dev_acc > prev_dev_acc and dev_acc > 0.67:
        prev_dev_acc = dev_acc
        print('New Best Accuracy: {}'.format(dev_acc))
        acc = int(100 * dev_acc)
        torch.save(model.state_dict(), checkpoint + '-a' + str(acc) + '.pt')
    print('Development accuracy for epoch {}: {}'.format(epoch + 1, correct / total))

torch.save(model.state_dict(), PATH + '-final.pt')

Initializing Model


  0%|          | 1/2994 [00:00<06:23,  7.81it/s]

Training started for epoch 1


 98%|█████████▊| 2922/2994 [01:58<00:02, 25.71it/s]

IndexError: ignored